This script evaluates the performance of different graph-aware architectures in a node classification problem. Several datasets are employed paying special attention to the homophily ratio.

In [6]:
import time
import numpy as np
from pandas import DataFrame
import matplotlib.pyplot as plt
import dgl
import networkx as nx
import torch
import torch.nn as nn


import utils
from gsp_utils.baselines_archs import GCNN_2L, MLP, GAT
from gsp_utils.baselines_models import NodeClassModel, GF_NodeClassModel
from gsp_utils.data import normalize_gso
from src.arch import GFGCN, GFGCNLayer, GFGCN_noh_Layer, GFGCN_Spows, Dual_GFGCN, NV_GFGCN, GCNN

# SEED = 0
SEED = 15
PATH = 'results/try_datasets'
SAVE = True
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

torch.manual_seed(SEED)

cuda:0


In [7]:
def print_full_results(accs, ellapsed_time, datasets, exps):
    mean_accs = accs.mean(axis=2)
    med_accs = np.median(accs, axis=2)
    std_accs = accs.std(axis=2)
    mean_t = ellapsed_time.mean(axis=2)

    for i, dataset_name in enumerate(datasets):
        graph = getattr(dgl.data, dataset_name)(verbose=False)[0]
        edge_hom = dgl.edge_homophily(graph, graph.ndata['label'])

        print(f'{dataset_name} (homophily ratio: {edge_hom:.3f})')
        for j, exp in enumerate(exps):
            print(f'\t- {exp["leg"]}:\tmean: {mean_accs[j,i]:.3f} - std: {std_accs[j,i]:.4f} - med: {med_accs[j,i]:.3f} - time: {mean_t[j,i]:.2f} mins')
        
        print()

def summary_table(accs, datasets, exps, median=False):
    mean_accs = accs.mean(axis=2)
    cols_name = []
    for dataset_name in datasets:
        graph = getattr(dgl.data, dataset_name)(verbose=False)[0]
        edge_hom = dgl.edge_homophily(graph, graph.ndata['label'])
        cols_name.append(f'{dataset_name} ({edge_hom:.2f})')

    index_name = [exp['leg'] for exp in exps]

    return DataFrame(mean_accs, columns=cols_name, index=index_name)


In [8]:
DATASETS = ['TexasDataset',  'WisconsinDataset', 'CornellDataset', 'ChameleonDataset', 'CoraGraphDataset', 'CiteseerGraphDataset']
# DATASETS = ['CiteseerGraphDataset', 'CoraGraphDataset'. 'ChameleonDataset', 'ActorDataset']
# DATASETS = ['TexasDataset', 'WisconsinDataset', 'CornellDataset', 'CoraGraphDataset']


## Best params

In [9]:
N_RUNS = 5
ACT = nn.ReLU()  # nn.ReLU()
LAST_ACT = nn.Softmax(dim=1)
LOSS_FN = nn.CrossEntropyLoss() #nn.NLLLoss()

EXPS = [
    {'model': 'Kipf', 'norm': 'both', 'epochs': 200, 'lr': .01, 'wd': 5e-4, 'drop': .5,
          'L': 2, 'hid_dim': 16, 'leg': 'Kipf-normA'},
        {'model': 'Kipf', 'norm': 'none', 'epochs': 200, 'lr': .01, 'wd': 5e-4, 'drop': .5,
          'L': 2, 'hid_dim': 16, 'leg': 'Kipf-A'},

        {'model': 'MLP', 'epochs': 200, 'lr': .01, 'wd': 5e-4, 'drop': .5, 'L': 2,
          'hid_dim': 16, 'leg': 'MLP'},
        {'model': 'MLP', 'epochs': 200, 'lr': .01, 'wd': 5e-4, 'drop': .5, 'L': 2,
          'hid_dim': 32, 'leg': 'MLP'},

        {'model': 'GAT', 'heads': 2, 'epochs': 200, 'lr': .01, 'wd': 5e-4, 'drop': 0,
          'hid_dim': 16, 'leg': 'GAT'},
          

        {'model': 'GFGCN', 'epochs': 200, 'e_h': 10, 'e_W': 10, 'lr': .005, 'wd': .001, 'drop': .25,
         'hid_dim': 50, 'L': 3, 'K': 2, 'h0': 1, 'norm': True, 'leg': 'A-GCN-normA'},

        {'model': 'GFGCN', 'epochs': 200, 'e_h': 25, 'e_W': 25, 'lr': .005, 'wd': .001, 'drop': .25,
         'hid_dim': 32,'L': 2, 'K': 3, 'h0': 1, 'norm': False, 'leg': 'A-GCN'},

        # {'model': 'GFGCN', 'epochs': 200, 'e_h': 25, 'e_W': 25, 'lr': .005, 'wd': .001, 'drop': .25,
        #  'hid_dim': 50,'L': 3, 'K': 2, 'h0': 1, 'norm': False, 'leg': 'A-GCN-v2'},

        {'model': 'H-GFGCN', 'epochs': 200, 'e_h': 5, 'e_W': 25, 'lr': .005, 'wd': 5e-4, 'drop': .5,
         'hid_dim': 32,'L': 3, 'K': 2, 'norm': True, 'leg': 'H-GCN-normH'},

        {'model': 'NV-GFGCN', 'epochs': 200, 'e_h': 5, 'e_W': 25, 'lr': .005, 'wd': .001, 'drop': .25,
         'hid_dim': 32, 'L': 2, 'K': 2, 'type': 'both', 'leg': 'NV-GCN-both'},
        
        # {'model': 'NV-GFGCN', 'epochs': 200, 'e_h': 5, 'e_W': 25, 'lr': .005, 'wd': .001, 'drop': .25,
        #  'hid_dim': 32, 'L': 2, 'K': 2, 'type': 'left', 'leg': 'NV-GCN-left'},

        # {'model': 'NV-GFGCN', 'epochs': 200, 'e_h': 5, 'e_W': 25, 'lr': .005, 'wd': .001, 'drop': .25,
        #  'hid_dim': 32, 'L': 2, 'K': 2, 'type': 'right', 'leg': 'NV-GCN-right'},

        {'model': 'Dual-GFGCN', 'epochs': 200, 'e_h': 50, 'e_W': 25, 'lr': .005, 'wd': .001, 'drop': .25,
         'bias': True, 'alpha': None, 'hid_dim': 32,'L': 3, 'K': 2, 'h0': 1, 'norm': False, 'leg': 'Dual-GFCN-alpha'},

        {'model': 'Dual-GFGCN', 'epochs': 200, 'e_h': 10, 'e_W': 10, 'lr': .005, 'wd': .001, 'drop': .25,
         'bias': True, 'alpha': None, 'hid_dim': 32,'L': 3, 'K': 2, 'h0': 1, 'norm': True, 'leg': 'Dual-GFCN-alpha-n'},

        {'model': 'Dual-GFGCN', 'epochs': 200, 'e_h': 25, 'e_W': 25, 'lr': .005, 'wd': .005, 'drop': .25,
          'bias': False, 'alpha': .8, 'hid_dim': 32,'L': 3, 'K': 2, 'h0': 1, 'norm': False, 'leg': 'Dual-GFCN'},

        {'model': 'Dual-GFGCN', 'epochs': 200, 'e_h': 25, 'e_W': 25, 'lr': .005, 'wd': .005, 'drop': .25,
          'bias': True, 'alpha': .8, 'hid_dim': 32,'L': 3, 'K': 2, 'h0': .1, 'norm': True, 'leg': 'Dual-GFCN-n'},

        # {'model': 'H-GFGCN', 'epochs': 200, 'e_h': 25, 'e_W': 25, 'lr': .005, 'wd': .001, 'drop': .25,
        #  'hid_dim': 32,'L': 2, 'K': 3, 'norm': False, 'leg': 'H-GCN'},
         

        {'model': 'noh-GFGCN', 'epochs': 500, 'lr': .005, 'wd': .01, 'drop': .25,
         'hid_dim': 32, 'L': 2, 'K': 2, 'norm': True, 'leg': 'W-GCN-normA'},
        {'model': 'noh-GFGCN', 'epochs': 500, 'lr': .005, 'wd': .01, 'drop': .25,
         'hid_dim': 32, 'L': 2, 'K': 2, 'norm': False, 'leg': 'W-GCN'},
      ]


In [ ]:
best_accs = np.zeros((len(EXPS), len(DATASETS), N_RUNS))
accs_best_val = np.zeros((len(EXPS), len(DATASETS), N_RUNS))
accs_best_val2 = np.zeros((len(EXPS), len(DATASETS), N_RUNS))
ellapsed_times = np.zeros((len(EXPS), len(DATASETS), N_RUNS))
for j, dataset in enumerate(DATASETS):

    print(dataset)

    for i in range(N_RUNS):
        print(f'{i}:', end=' ')
        
        A, feat, labels, n_class, masks = utils.get_data_dgl(dataset, dev=device, idx=i%10)
        N = A.shape[0]
        in_dim = feat.shape[1]
        out_dim = n_class
        
        for k, exp in enumerate(EXPS):
            t_i = time.time()
            if exp['model'] == 'Kipf':
                arch = GCNN(in_dim, exp['hid_dim'], out_dim, exp['L'], act=ACT, last_act=nn.Identity(),
                            dropout=exp['drop'])
                if exp['norm'] != 'none':
                    S = torch.Tensor(normalize_gso(A.copy(), exp['norm'])).to(device)
                else:
                    S = torch.Tensor(A.copy()).to(device)
            elif exp['model'] == 'MLP':
                arch = MLP(in_dim,  exp['hid_dim'], out_dim, exp['L'], act=ACT, last_act=nn.Identity(), dropout=exp['drop'])

            elif exp['model'] == 'GAT':
                gat_params = {'attn_drop': exp['drop']}
                arch = GAT(in_dim,  exp['hid_dim'], out_dim, exp['heads'], gat_params, act=ACT, last_act=LAST_ACT)
                S = dgl.from_networkx(nx.from_numpy_array(A)).add_self_loop().to(device)

            elif exp['model'] == 'GFGCN':
                arch = GFGCN(in_dim, exp['hid_dim'], out_dim, exp['L'], exp['K'], act=ACT, last_act=LAST_ACT,
                         dropout=exp['drop'], diff_layer=GFGCNLayer, init_h0=exp['h0'])
                if exp['norm']:
                    S = torch.Tensor(normalize_gso(A, 'both', add_id=False)).to(device)
                else:
                    S = torch.Tensor(A).to(device)

            elif exp['model'] == 'H-GFGCN':
                arch = GFGCN_Spows(in_dim, exp['hid_dim'], out_dim, exp['L'], exp['K'], act=ACT, last_act=LAST_ACT,
                                   dropout=exp['drop'], norm=exp['norm'], dev=device)
                S = torch.Tensor(A).to(device)

            elif exp['model'] == 'NV-GFGCN':
                arch = NV_GFGCN(in_dim, exp['hid_dim'], out_dim, exp['L'], exp['K'], N, act=ACT,
                                last_act=LAST_ACT, f_type=exp['type'], dropout=exp['drop'])
                S = torch.Tensor(A).to(device)
            
            elif exp['model'] == 'noh-GFGCN':
                arch = GFGCN(in_dim, exp['hid_dim'], out_dim, exp['L'], exp['K'], act=ACT, last_act=LAST_ACT,
                             dropout=exp['drop'], diff_layer=GFGCN_noh_Layer)
                if exp['norm']:
                    S = torch.Tensor(normalize_gso(A, 'both', add_id=False)).to(device)
                else:
                    S = torch.Tensor(A).to(device)

            elif exp['model'] == 'Dual-GFGCN':
                arch = Dual_GFGCN(in_dim, exp['hid_dim'], out_dim, exp['L'], exp['K'], act=ACT, alpha=exp['alpha'],
                                  last_act=LAST_ACT, dropout=exp['drop'], init_h0=exp['h0'], bias=exp['bias'])
                if exp['norm']:
                    S = torch.Tensor(normalize_gso(A, 'both', add_id=False)).to(device)
                else:
                    S = torch.Tensor(A).to(device)
            else:
                raise Exception(f'ERROR: unknown architecture {exp["model"]}')

            if exp['model'] in ['Kipf', 'MLP', 'GAT', 'noh-GFGCN']:
                model = NodeClassModel(arch, S, masks, LOSS_FN, device=device)
                loss, acc = model.train(feat, labels, exp['epochs'], exp['lr'], exp['wd'])
            else:
                model = GF_NodeClassModel(arch, S, exp['K'], masks, LOSS_FN, device=device)
                loss, acc = model.train(feat, labels, exp['epochs'], exp['lr'], exp['wd'],
                                        epochs_h=exp['e_h'], epochs_W=exp['e_W'])
            ellapsed_t = (time.time()-t_i)/60
            
            
            best_accs[k,j,i] = np.max(acc["test"])
            accs_best_val[k,j,i] = model.test(feat, model.S, labels, masks['test'])
            accs_best_val2[k,j,i] = acc["test"][np.argmax(acc["val"])]

            ellapsed_times[k,j,i] = ellapsed_t

            print(f'{accs_best_val[k,j,i]:.3f} ({best_accs[k,j,i]:.3f})', end=' - ')
        print()      
    print()


print_full_results(accs_best_val, ellapsed_times, DATASETS, EXPS)
table_acc1 = summary_table(best_accs, DATASETS, EXPS)
table_acc_val1 = summary_table(accs_best_val, DATASETS, EXPS)
table_acc_val1b = summary_table(accs_best_val2, DATASETS, EXPS)

TexasDataset
0: 0.622 (0.676) - 0.649 (0.676) - 0.757 (0.784) - 0.784 (0.811) - 0.514 (0.649) - 

/home/srey/Investigacion/robust_minmax_gnn/gsp_utils/data.py:44: RuntimeWarning: divide by zero encountered in divide
  D_inv = np.diag(np.where(np.isclose(deg_vec, 0), 0, 1/deg_vec))


0.838 (0.838) - 0.838 (0.892) - 0.838 (0.919) - 0.622 (0.676) - 0.865 (0.892) - 0.811 (0.865) - 0.838 (0.892) - 0.784 (0.838) - 0.811 (0.838) - 0.784 (0.892) - 
1: 0.568 (0.622) - 0.703 (0.730) - 0.703 (0.757) - 0.811 (0.919) - 0.622 (0.649) - 0.919 (0.973) - 0.973 (0.973) - 0.973 (0.973) - 0.649 (0.676) - 0.865 (0.919) - 0.757 (0.919) - 0.973 (0.973) - 0.892 (0.946) - 0.838 (0.919) - 0.757 (0.865) - 
2: 0.486 (0.541) - 0.486 (0.541) - 0.703 (0.730) - 0.757 (0.784) - 0.514 (0.622) - 0.811 (0.865) - 0.838 (0.919) - 0.919 (0.919) - 0.595 (0.649) - 0.784 (0.892) - 0.838 (0.865) - 0.838 (0.892) - 0.784 (0.892) - 0.730 (0.757) - 0.730 (0.784) - 
3: 0.622 (0.649) - 0.595 (0.622) - 0.784 (0.838) - 0.811 (0.892) - 0.649 (0.676) - 0.838 (0.946) - 0.811 (0.892) - 0.865 (0.946) - 0.622 (0.676) - 0.892 (0.919) - 0.919 (0.946) - 0.892 (0.946) - 0.865 (0.946) - 0.811 (0.865) - 0.757 (0.784) - 
4: 0.514 (0.568) - 0.595 (0.622) - 0.784 (0.811) - 0.730 (0.838) - 0.514 (0.649) - 0.865 (0.892) - 0.811 (0

In [ ]:
if SAVE:
    timestr = time.strftime("%Y%m%d-%H%M")
    table_acc_val1.to_csv(PATH + 'best_params_val_test_' + timestr)
    table_acc_val1.to_csv(PATH + 'best_params_best_val_' + timestr)

In [15]:
table_acc_val1

,TexasDataset (0.11),WisconsinDataset (0.20),CornellDataset (0.13),ChameleonDataset (0.24),CoraGraphDataset (0.81),CiteseerGraphDataset (0.74)
Kipf-normA,0.454054,0.450980,0.318919,0.313596,0.8144,0.7086
Kipf-A,0.583784,0.509804,0.459459,0.250000,0.7066,0.5864
MLP,0.789189,0.823529,0.740541,0.485965,0.5658,0.5704
MLP,0.810811,0.831373,0.745946,0.496053,0.5896,0.5840
GAT,0.578378,0.525490,0.421622,0.617982,0.5912,0.6024
A-GCN-normA,0.864865,0.862745,0.756757,0.440351,0.8034,0.6900
A-GCN,0.859459,0.866667,0.756757,0.447807,0.7612,0.6620
H-GCN-normH,0.854054,0.862745,0.751351,0.443421,0.8030,0.6876
NV-GCN-both,0.551351,0.505882,0.513514,0.436842,0.7678,0.6324
Dual-GFCN-alpha,0.859459,0.874510,0.762162,0.457456,0.7670,0.6766


In [16]:
table_acc_val1b

,TexasDataset (0.11),WisconsinDataset (0.20),CornellDataset (0.13),ChameleonDataset (0.24),CoraGraphDataset (0.81),CiteseerGraphDataset (0.74)
Kipf-normA,0.540541,0.474510,0.437838,0.314035,0.8122,0.7076
Kipf-A,0.572973,0.509804,0.432432,0.251316,0.7348,0.5912
MLP,0.789189,0.819608,0.751351,0.505263,0.5618,0.5670
MLP,0.821622,0.854902,0.751351,0.509649,0.5742,0.5846
GAT,0.578378,0.525490,0.410811,0.613596,0.5926,0.6244
A-GCN-normA,0.843243,0.862745,0.783784,0.459649,0.7964,0.6906
A-GCN,0.783784,0.870588,0.767568,0.442982,0.7588,0.6620
H-GCN-normH,0.848649,0.819608,0.745946,0.439474,0.8044,0.6934
NV-GCN-both,0.589189,0.509804,0.508108,0.435965,0.7676,0.6406
Dual-GFCN-alpha,0.881081,0.858824,0.762162,0.446053,0.7658,0.6782


## Initial Params

In [9]:
# BEST PARAMETERS
## Reaining params
N_RUNS = 10
N_EPOCHS = 200  # 500
LR = .01
WD = 5e-4
DROPOUT = .5

# BEST PARAMETERS
## Architecture params
N_LAYERS = 2
K = 3
HID_DIM = 16

## Model params
h0 = 1
ACT = nn.ReLU()
LAST_ACT = nn.LogSoftmax(dim=1)
LOSS_FN = nn.NLLLoss()

EXPS = [{'model': 'Kipf', 'norm': 'both', 'epochs': 200, 'lr': .01, 'wd': 5e-4, 'drop': .5,
          'L': 2, 'hid_dim': 16, 'leg': 'Kipf-normA'},
        {'model': 'Kipf', 'norm': 'none', 'epochs': 200, 'lr': .01, 'wd': 5e-4, 'drop': .5,
          'L': 2, 'hid_dim': 16, 'leg': 'Kipf-A'},

        # {'model': 'MLP', 'epochs': 200, 'lr': .01, 'wd': 5e-4, 'drop': .5,
        #   'hid_dim': 16, 'leg': 'MLP'},
        {'model': 'MLP', 'epochs': 200, 'lr': .01, 'wd': 5e-4, 'drop': .5,
          'hid_dim': 16, 'leg': 'MLP'},

        {'model': 'GAT', 'heads': 2, 'epochs': 200, 'lr': .01, 'wd': 5e-4, 'drop': 0,
          'hid_dim': 16, 'leg': 'GAT'},
          

        {'model': 'GFGCN', 'epochs': 200, 'e_h': 10, 'e_W': 10, 'lr': .005, 'wd': .001, 'drop': .25,
         'hid_dim': 50, 'L': 3, 'K': 2, 'h0': 1, 'norm': True, 'leg': 'A-GCN-normA'},

        {'model': 'GFGCN', 'epochs': 200, 'e_h': 25, 'e_W': 25, 'lr': .005, 'wd': .001, 'drop': .25,
         'hid_dim': 32,'L': 2, 'K': 3, 'h0': 1, 'norm': False, 'leg': 'A-GCN'},

        {'model': 'GFGCN', 'epochs': 200, 'e_h': 25, 'e_W': 25, 'lr': .005, 'wd': .001, 'drop': .25,
         'hid_dim': 50,'L': 3, 'K': 2, 'h0': 1, 'norm': False, 'leg': 'A-GCN-v2'},

        {'model': 'H-GFGCN', 'epochs': 200, 'e_h': 5, 'e_W': 25, 'lr': .005, 'wd': 5e-4, 'drop': .5,
         'hid_dim': 32,'L': 3, 'K': 2, 'norm': True, 'leg': 'H-GCN-normH'},

        {'model': 'Dual-GFGCN', 'epochs': 200, 'e_h': 50, 'e_W': 25, 'lr': .005, 'wd': .001, 'drop': .25,
         'bias': True, 'alpha': None, 'hid_dim': 32,'L': 3, 'K': 2, 'h0': 1, 'norm': False, 'leg': 'Dual-GFCN-alpha'},

        {'model': 'Dual-GFGCN', 'epochs': 200, 'e_h': 10, 'e_W': 10, 'lr': .005, 'wd': .001, 'drop': .25,
         'bias': True, 'alpha': None, 'hid_dim': 32,'L': 3, 'K': 2, 'h0': 1, 'norm': True, 'leg': 'Dual-GFCN-alpha-n'},

        {'model': 'Dual-GFGCN', 'epochs': 200, 'e_h': 25, 'e_W': 25, 'lr': .005, 'wd': .005, 'drop': .25,
          'bias': False, 'alpha': .8, 'hid_dim': 32,'L': 3, 'K': 2, 'h0': 1, 'norm': False, 'leg': 'Dual-GFCN'},

        {'model': 'Dual-GFGCN', 'epochs': 200, 'e_h': 25, 'e_W': 25, 'lr': .005, 'wd': .005, 'drop': .25,
          'bias': True, 'alpha': .8, 'hid_dim': 32,'L': 3, 'K': 2, 'h0': .1, 'norm': True, 'leg': 'Dual-GFCN-n'},

        # {'model': 'H-GFGCN', 'epochs': 200, 'e_h': 25, 'e_W': 25, 'lr': .005, 'wd': .001, 'drop': .25,
        #  'hid_dim': 32,'L': 2, 'K': 3, 'norm': False, 'leg': 'H-GCN'},
         

        {'model': 'noh-GFGCN', 'epochs': 500, 'lr': .005, 'wd': .01, 'drop': .25,
         'hid_dim': 32, 'L': 2, 'K': 2, 'norm': True, 'leg': 'W-GCN-normA'},
        {'model': 'noh-GFGCN', 'epochs': 500, 'lr': .005, 'wd': .01, 'drop': .25,
         'hid_dim': 32, 'L': 2, 'K': 2, 'norm': False, 'leg': 'W-GCN'}]

In [10]:
best_accs = np.zeros((len(EXPS), len(DATASETS), N_RUNS))
accs_best_val = np.zeros((len(EXPS), len(DATASETS), N_RUNS))
accs_best_val2 = np.zeros((len(EXPS), len(DATASETS), N_RUNS))
ellapsed_times = np.zeros((len(EXPS), len(DATASETS), N_RUNS))
for j, dataset in enumerate(DATASETS):

    print(dataset)

    for i in range(N_RUNS):
        print(f'{i}:', end=' ')
        
        A, feat, labels, n_class, masks = utils.get_data_dgl(dataset, dev=device, idx=i%10)
        N = A.shape[0]
        in_dim = feat.shape[1]
        out_dim = n_class
        
        for k, exp in enumerate(EXPS):
            t_i = time.time()
            if exp['model'] == 'Kipf':
                arch = GCNN_2L(in_dim, exp['hid_dim'], out_dim, act=ACT, last_act=LAST_ACT,
                               dropout=exp['drop'], norm=exp['norm'])
                S = dgl.from_networkx(nx.from_numpy_array(A)).add_self_loop().to(device)

            elif exp['model'] == 'MLP':
                arch = MLP(in_dim,  exp['hid_dim'], out_dim, dropout=exp['drop'])

            elif exp['model'] == 'GAT':
                gat_params = {'attn_drop': exp['drop']}
                arch = GAT(in_dim,  exp['hid_dim'], out_dim, exp['heads'], gat_params, act=ACT, last_act=LAST_ACT)

            elif exp['model'] == 'GFGCN':
                arch = GFGCN(in_dim, exp['hid_dim'], out_dim, exp['L'], exp['K'], act=ACT, last_act=LAST_ACT,
                         dropout=exp['drop'], diff_layer=GFGCNLayer, init_h0=exp['h0'])
                if exp['norm']:
                    S = torch.Tensor(normalize_gso(A, 'both')).to(device)
                else:
                    S = torch.Tensor(A).to(device)

            elif exp['model'] == 'H-GFGCN':
                arch = GFGCN_Spows(in_dim, exp['hid_dim'], out_dim, exp['L'], exp['K'], act=ACT, last_act=LAST_ACT,
                                   dropout=exp['drop'], norm=exp['norm'], dev=device)
                S = torch.Tensor(A).to(device)

            elif exp['model'] == 'noh-GFGCN':
                arch = GFGCN(in_dim, exp['hid_dim'], out_dim, exp['L'], exp['K'], act=ACT, last_act=LAST_ACT,
                             dropout=exp['drop'], diff_layer=GFGCN_noh_Layer)
                if exp['norm']:
                    S = torch.Tensor(normalize_gso(A, 'both')).to(device)
                else:
                    S = torch.Tensor(A).to(device)

            elif exp['model'] == 'Dual-GFGCN':
                arch = Dual_GFGCN(in_dim, exp['hid_dim'], out_dim, exp['L'], exp['K'], act=ACT, alpha=exp['alpha'],
                                  last_act=LAST_ACT, dropout=exp['drop'], init_h0=exp['h0'], bias=exp['bias'])
                if exp['norm']:
                    S = torch.Tensor(normalize_gso(A, 'both')).to(device)
                else:
                    S = torch.Tensor(A).to(device)
            else:
                raise Exception(f'ERROR: unknown architecture {exp["model"]}')

            if exp['model'] in ['Kipf', 'MLP', 'GAT', 'noh-GFGCN']:
                model = NodeClassModel(arch, S, masks, LOSS_FN, device=device)
                loss, acc = model.train(feat, labels, exp['epochs'], exp['lr'], exp['wd'])
            else:
                model = GF_NodeClassModel(arch, S, exp['K'], masks, LOSS_FN, device=device)
                loss, acc = model.train(feat, labels, exp['epochs'], exp['lr'], exp['wd'],
                                        epochs_h=exp['e_h'], epochs_W=exp['e_W'])
            ellapsed_t = (time.time()-t_i)/60
            
            
            best_accs[k,j,i] = np.max(acc["test"])
            accs_best_val[k,j,i] = model.test(feat, model.S, labels, masks['test'])
            accs_best_val2[k,j,i] = acc["test"][np.argmax(acc["val"])]

            ellapsed_times[k,j,i] = ellapsed_t

            print(f'{accs_best_val[k,j,i]:.3f} ({best_accs[k,j,i]:.3f})', end=' - ')
        print()      
    print()


print_full_results(accs_best_val, ellapsed_times, DATASETS, EXPS)
table_acc2 = summary_table(best_accs, DATASETS, EXPS)
table_acc_val2 = summary_table(accs_best_val, DATASETS, EXPS)
table_acc_val2b = summary_table(accs_best_val2, DATASETS, EXPS)

TexasDataset
0: 0.649 (0.649) - 0.189 (0.703) - 0.135 (0.432) - 0.622 (0.649) - 

/home/srey/Investigacion/robust_minmax_gnn/gsp_utils/data.py:44: RuntimeWarning: divide by zero encountered in divide
  D_inv = np.diag(np.where(np.isclose(deg_vec, 0), 0, 1/deg_vec))


0.351 (0.649) - 0.730 (0.811) - 0.378 (0.514) - 0.541 (0.595) - 0.541 (0.568) - 0.541 (0.649) - 0.838 (0.865) - 0.838 (0.865) - 0.838 (0.838) - 0.838 (0.892) - 
1: 0.595 (0.595) - 0.595 (0.595) - 0.595 (0.757) - 0.514 (0.649) - 0.622 (0.784) - 0.676 (0.703) - 0.595 (0.622) - 0.568 (0.865) - 0.568 (0.622) - 0.676 (0.757) - 0.865 (0.892) - 0.892 (0.973) - 0.865 (0.892) - 0.784 (0.838) - 
2: 0.486 (0.541) - 0.486 (0.568) - 0.486 (0.649) - 0.459 (0.568) - 0.432 (0.622) - 0.622 (0.703) - 0.432 (0.486) - 0.459 (0.541) - 0.351 (0.568) - 0.784 (0.811) - 0.811 (0.892) - 0.784 (0.892) - 0.730 (0.757) - 0.784 (0.784) - 
3: 0.622 (0.622) - 0.622 (0.622) - 0.622 (0.649) - 0.622 (0.649) - 0.865 (0.919) - 0.405 (0.514) - 0.486 (0.541) - 0.541 (0.676) - 0.378 (0.541) - 0.486 (0.676) - 0.838 (0.892) - 0.838 (0.946) - 0.838 (0.838) - 0.730 (0.784) - 
4: 0.541 (0.595) - 0.541 (0.622) - 0.162 (0.243) - 0.568 (0.622) - 0.730 (0.811) - 0.486 (0.541) - 0.784 (0.811) - 0.568 (0.622) - 0.486 (0.568) - 0.622 (0

In [11]:
table_acc_val2

,TexasDataset (0.11),WisconsinDataset (0.20),CornellDataset (0.13),ChameleonDataset (0.24),CoraGraphDataset (0.81),CiteseerGraphDataset (0.74)
Kipf-normA,0.583784,0.515686,0.456757,0.630044,0.8139,0.7085
Kipf-A,0.537838,0.492157,0.451351,0.276535,0.7699,0.6637
MLP,0.497297,0.415686,0.427027,0.218421,0.1295,0.1674
GAT,0.572973,0.539216,0.440541,0.567325,0.7491,0.6610
A-GCN-normA,0.648649,0.607843,0.486486,0.379825,0.7869,0.6339
A-GCN,0.548649,0.458824,0.470270,0.252193,0.6789,0.5861
A-GCN-v2,0.551351,0.456863,0.456757,0.282018,0.6852,0.5319
H-GCN-normH,0.510811,0.488235,0.375676,0.319956,0.8087,0.6870
Dual-GFCN-alpha,0.459459,0.503922,0.400000,0.494956,0.6829,0.5196
Dual-GFCN-alpha-n,0.605405,0.605882,0.575676,0.411404,0.7911,0.6670


In [12]:
table_acc_val2b

,TexasDataset (0.11),WisconsinDataset (0.20),CornellDataset (0.13),ChameleonDataset (0.24),CoraGraphDataset (0.81),CiteseerGraphDataset (0.74)
Kipf-normA,0.562162,0.535294,0.464865,0.637061,0.8128,0.7088
Kipf-A,0.597297,0.519608,0.435135,0.314474,0.7686,0.6658
MLP,0.564865,0.556863,0.486486,0.298026,0.2169,0.2622
GAT,0.581081,0.515686,0.448649,0.655702,0.7918,0.6913
A-GCN-normA,0.718919,0.668627,0.570270,0.464254,0.7945,0.6861
A-GCN,0.589189,0.488235,0.532432,0.269298,0.6990,0.6247
A-GCN-v2,0.562162,0.468627,0.545946,0.320175,0.7545,0.6404
H-GCN-normH,0.616216,0.696078,0.516216,0.316447,0.8117,0.6880
Dual-GFCN-alpha,0.502703,0.547059,0.535135,0.541447,0.7566,0.6392
Dual-GFCN-alpha-n,0.670270,0.729412,0.624324,0.450877,0.7915,0.6869


# Best Params Cora

In [18]:
# BEST PARAMETERS
## Reaining params
N_RUNS = 10
N_EPOCHS = 200  # 500
LR = .01
WD = 5e-4
DROPOUT = .5

# BEST PARAMETERS
## Architecture params
N_LAYERS = 2
K = 3
HID_DIM = 16

## Model params
h0 = 1
ACT = nn.ReLU()
LAST_ACT = nn.Identity() # nn.LogSoftmax(dim=1)
LOSS_FN = nn.CrossEntropyLoss()

EXPS = [{'model': 'Kipf', 'norm': 'both', 'epochs': 200, 'lr': .01, 'wd': 5e-4, 'drop': .5,
          'L': 2, 'hid_dim': 64, 'leg': 'Kipf-normA'},
        {'model': 'Kipf', 'norm': 'none', 'epochs': 200, 'lr': .01, 'wd': 5e-4, 'drop': .5,
          'L': 2, 'hid_dim': 64, 'leg': 'Kipf-A'},

        # {'model': 'MLP', 'epochs': 200, 'lr': .01, 'wd': 5e-4, 'drop': .5,
        #   'hid_dim': 16, 'leg': 'MLP'},
        {'model': 'MLP', 'epochs': 200, 'lr': .01, 'wd': 5e-4, 'drop': 0.,
          'hid_dim': 16, 'L': 3, 'leg': 'MLP'},

        {'model': 'GAT', 'heads': 2, 'epochs': 200, 'lr': .01, 'wd': 5e-4, 'drop': 0,
          'hid_dim': 16, 'leg': 'GAT'},
          

        # {'model': 'GFGCN', 'epochs': 200, 'e_h': 10, 'e_W': 10, 'lr': .01, 'wd': .001, 'drop': .5,
        #  'hid_dim': 50, 'L': 2, 'K': 3, 'h0': 1, 'norm': True, 'last_act': nn.LogSoftmax(dim=1), 'leg': 'A-GCN-normA'},

        {'model': 'GFGCN', 'epochs': 200, 'e_h': 25, 'e_W': 25, 'lr': .01, 'wd': .001, 'drop': .0,
         'hid_dim': 64,'L': 2, 'K': 3, 'h0': 1, 'norm': True, 'act': nn.ReLU(), 'last_act': nn.LogSoftmax(dim=1), 'leg': 'A-GCN'},

        # {'model': 'GFGCN', 'epochs': 200, 'e_h': 25, 'e_W': 25, 'lr': .005, 'wd': .001, 'drop': .5,
        #  'hid_dim': 64,'L': 2, 'K': 3, 'h0': 1, 'norm': False, 'leg': 'A-GCN-v2'},

        {'model': 'H-GFGCN', 'epochs': 200, 'e_h': 5, 'e_W': 25, 'lr': .001, 'wd': 5e-4, 'drop': .5,
         'hid_dim': 32, 'L': 3, 'K': 2, 'norm': True, 'leg': 'H-GCN-normH', 'act': nn.ReLU(), 'last_act': nn.Identity()},

        # {'model': 'Dual-GFGCN', 'epochs': 200, 'e_h': 50, 'e_W': 25, 'lr': .005, 'wd': .001, 'drop': .25,
        #  'bias': True, 'alpha': None, 'hid_dim': 32,'L': 3, 'K': 2, 'h0': 1, 'norm': False, 'leg': 'Dual-GFCN-alpha'},

        {'model': 'Dual-GFGCN', 'epochs': 200, 'e_h': 10, 'e_W': 10, 'lr': .01, 'wd': .005, 'drop': .5,
         'bias': True, 'alpha': None, 'hid_dim': 64,'L': 2, 'K': 3, 'h0': 1, 'norm': True,
         'act': nn.Tanh(), 'last_act': nn.LogSoftmax(dim=1), 'leg': 'Dual-GFCN-alpha-n'},

        # {'model': 'Dual-GFGCN', 'epochs': 200, 'e_h': 25, 'e_W': 25, 'lr': .005, 'wd': .005, 'drop': .25,
        #   'bias': False, 'alpha': .8, 'hid_dim': 32,'L': 3, 'K': 2, 'h0': 1, 'norm': False, 'leg': 'Dual-GFCN'},

        # {'model': 'Dual-GFGCN', 'epochs': 200, 'e_h': 25, 'e_W': 25, 'lr': .005, 'wd': .005, 'drop': .25,
        #   'bias': True, 'alpha': .8, 'hid_dim': 32,'L': 3, 'K': 2, 'h0': .1, 'norm': True, 'leg': 'Dual-GFCN-n'},

        # {'model': 'H-GFGCN', 'epochs': 200, 'e_h': 25, 'e_W': 25, 'lr': .005, 'wd': .001, 'drop': .25,
        #  'hid_dim': 32,'L': 2, 'K': 3, 'norm': False, 'leg': 'H-GCN'},
         

        {'model': 'noh-GFGCN', 'epochs': 500, 'lr': .01, 'wd': .01, 'drop': .0,
         'hid_dim': 64, 'L': 2, 'K': 3, 'norm': True, 'act': nn.Tanh(), 'last_act': nn.Identity(), 'leg': 'W-GCN'},
        # {'model': 'noh-GFGCN', 'epochs': 500, 'lr': .005, 'wd': .01, 'drop': .25,
        #  'hid_dim': 32, 'L': 2, 'K': 2, 'norm': False, 'leg': 'W-GCN'}
]

In [19]:
best_accs = np.zeros((len(EXPS), len(DATASETS), N_RUNS))
accs_best_val = np.zeros((len(EXPS), len(DATASETS), N_RUNS))
accs_best_val2 = np.zeros((len(EXPS), len(DATASETS), N_RUNS))
ellapsed_times = np.zeros((len(EXPS), len(DATASETS), N_RUNS))
for j, dataset in enumerate(DATASETS):

    print(dataset)

    for i in range(N_RUNS):
        print(f'{i}:', end=' ')
        
        A, feat, labels, n_class, masks = utils.get_data_dgl(dataset, dev=device, idx=i%10)
        N = A.shape[0]
        in_dim = feat.shape[1]
        out_dim = n_class
        
        for k, exp in enumerate(EXPS):
            t_i = time.time()
            if exp['model'] == 'Kipf':
                arch = GCNN(in_dim, exp['hid_dim'], out_dim, exp['L'], act=ACT, last_act=LAST_ACT,
                            dropout=exp['drop'])
                if exp['norm'] != 'none':
                    S = torch.Tensor(normalize_gso(A.copy(), exp['norm'])).to(device)
                else:
                    S = torch.Tensor(A.copy()).to(device)
            elif exp['model'] == 'MLP':
                arch = MLP(in_dim,  exp['hid_dim'], out_dim, exp['L'], act=ACT, last_act=LAST_ACT, dropout=exp['drop'])

            elif exp['model'] == 'GAT':
                gat_params = {'attn_drop': exp['drop']}
                arch = GAT(in_dim,  exp['hid_dim'], out_dim, exp['heads'], gat_params, act=ACT, last_act=LAST_ACT)
                S = dgl.from_networkx(nx.from_numpy_array(A)).add_self_loop().to(device)

            elif exp['model'] == 'GFGCN':
                arch = GFGCN(in_dim, exp['hid_dim'], out_dim, exp['L'], exp['K'], act=exp['act'], last_act=exp['last_act'],
                         dropout=exp['drop'], diff_layer=GFGCNLayer, init_h0=exp['h0'])
                if exp['norm']:
                    S = torch.Tensor(normalize_gso(A, 'both', add_id=False)).to(device)
                else:
                    S = torch.Tensor(A).to(device)

            elif exp['model'] == 'H-GFGCN':
                arch = GFGCN_Spows(in_dim, exp['hid_dim'], out_dim, exp['L'], exp['K'], act=exp['act'], last_act=exp['last_act'],
                                   dropout=exp['drop'], norm=exp['norm'], dev=device)
                S = torch.Tensor(A).to(device)

            elif exp['model'] == 'noh-GFGCN':
                arch = GFGCN(in_dim, exp['hid_dim'], out_dim, exp['L'], exp['K'], act=exp['act'], last_act=exp['last_act'],
                             dropout=exp['drop'], diff_layer=GFGCN_noh_Layer)
                if exp['norm']:
                    S = torch.Tensor(normalize_gso(A, 'both', add_id=False)).to(device)
                else:
                    S = torch.Tensor(A).to(device)

            elif exp['model'] == 'Dual-GFGCN':
                arch = Dual_GFGCN(in_dim, exp['hid_dim'], out_dim, exp['L'], exp['K'], act=exp['act'], alpha=exp['alpha'],
                                  last_act=LAST_ACT, dropout=exp['drop'], init_h0=exp['h0'], bias=exp['bias'])
                if exp['norm']:
                    S = torch.Tensor(normalize_gso(A, 'both', add_id=False)).to(device)
                else:
                    S = torch.Tensor(A).to(device)
            else:
                raise Exception(f'ERROR: unknown architecture {exp["model"]}')

            if exp['model'] in ['Kipf', 'MLP', 'GAT', 'noh-GFGCN']:
                model = NodeClassModel(arch, S, masks, LOSS_FN, device=device)
                loss, acc = model.train(feat, labels, exp['epochs'], exp['lr'], exp['wd'])
            else:
                model = GF_NodeClassModel(arch, S, exp['K'], masks, LOSS_FN, device=device)
                loss, acc = model.train(feat, labels, exp['epochs'], exp['lr'], exp['wd'],
                                        epochs_h=exp['e_h'], epochs_W=exp['e_W'])
            ellapsed_t = (time.time()-t_i)/60
            
            
            best_accs[k,j,i] = np.max(acc["test"])
            accs_best_val[k,j,i] = model.test(feat, model.S, labels, masks['test'])
            accs_best_val2[k,j,i] = acc["test"][np.argmax(acc["val"])]

            ellapsed_times[k,j,i] = ellapsed_t

            print(f'{accs_best_val[k,j,i]:.3f} ({best_accs[k,j,i]:.3f})', end=' - ')
        print()      
    print()


print_full_results(accs_best_val, ellapsed_times, DATASETS, EXPS)
table_acc2 = summary_table(best_accs, DATASETS, EXPS)
table_acc_val2 = summary_table(accs_best_val, DATASETS, EXPS)
table_acc_val2b = summary_table(accs_best_val2, DATASETS, EXPS)

TexasDataset
0: 0.486 (0.649) - 0.568 (0.622) - 0.811 (0.838) - 0.649 (0.649) - 0.757 (0.784) - 0.514 (0.568) - 0.784 (0.811) - 0.784 (0.838) - 
1: 0.595 (0.595) - 0.676 (0.676) - 0.811 (0.865) - 0.595 (0.649) - 0.919 (0.919) - 0.541 (0.622) - 0.865 (1.000) - 0.757 (0.784) - 
2: 0.270 (0.486) - 0.514 (0.595) - 0.703 (0.730) - 0.486 (0.486) - 0.757 (0.757) - 0.405 (0.432) - 0.811 (0.865) - 0.703 (0.757) - 
3: 0.541 (0.595) - 0.622 (0.622) - 0.865 (0.865) - 0.622 (0.649) - 0.865 (0.865) - 0.568 (0.595) - 0.757 (0.919) - 0.784 (0.784) - 
4: 0.486 (0.595) - 0.595 (0.595) - 0.757 (0.838) - 0.568 (0.595) - 0.757 (0.757) - 0.486 (0.541) - 0.838 (0.919) - 0.757 (0.838) - 
5: 0.568 (0.676) - 0.541 (0.568) - 0.811 (0.811) - 0.595 (0.649) - 0.838 (0.865) - 0.541 (0.622) - 0.838 (0.865) - 0.811 (0.838) - 
6: 0.541 (0.568) - 0.568 (0.622) - 0.784 (0.811) - 0.486 (0.568) - 0.649 (0.676) - 0.486 (0.514) - 0.757 (0.892) - 0.757 (0.784) - 
7: 0.568 (0.649) - 0.541 (0.595) - 0.649 (0.811) - 0.568 (0.622

ChameleonDataset (homophily ratio: 0.235)
	- Kipf-normA:	mean: 0.326 - std: 0.0280 - med: 0.333 - time: 0.01 mins
	- Kipf-A:	mean: 0.269 - std: 0.0237 - med: 0.272 - time: 0.01 mins
	- MLP:	mean: 0.434 - std: 0.0316 - med: 0.442 - time: 0.01 mins
	- GAT:	mean: 0.577 - std: 0.0466 - med: 0.584 - time: 0.04 mins
	- A-GCN:	mean: 0.342 - std: 0.0416 - med: 0.336 - time: 0.18 mins
	- H-GCN-normH:	mean: 0.308 - std: 0.0202 - med: 0.314 - time: 0.93 mins
	- Dual-GFCN-alpha-n:	mean: 0.597 - std: 0.0195 - med: 0.598 - time: 0.12 mins
	- W-GCN:	mean: 0.473 - std: 0.0178 - med: 0.478 - time: 0.04 mins

CoraGraphDataset (homophily ratio: 0.810)
	- Kipf-normA:	mean: 0.814 - std: 0.0016 - med: 0.813 - time: 0.01 mins
	- Kipf-A:	mean: 0.722 - std: 0.0355 - med: 0.734 - time: 0.01 mins
	- MLP:	mean: 0.448 - std: 0.0600 - med: 0.446 - time: 0.01 mins
	- GAT:	mean: 0.758 - std: 0.0133 - med: 0.762 - time: 0.05 mins
	- A-GCN:	mean: 0.802 - std: 0.0068 - med: 0.800 - time: 0.22 mins
	- H-GCN-normH:	mean: 

In [20]:
table_acc_val2

,TexasDataset (0.11),WisconsinDataset (0.20),CornellDataset (0.13),ChameleonDataset (0.24),CoraGraphDataset (0.81),CiteseerGraphDataset (0.74)
Kipf-normA,0.521622,0.466667,0.397297,0.326316,0.8135,0.7150
Kipf-A,0.597297,0.486275,0.467568,0.268640,0.7223,0.5726
MLP,0.772973,0.811765,0.718919,0.434430,0.4480,0.4935
GAT,0.578378,0.537255,0.416216,0.576754,0.7583,0.6454
A-GCN,0.797297,0.839216,0.713514,0.342325,0.8025,0.6972
H-GCN-normH,0.513514,0.466667,0.391892,0.307675,0.8060,0.6825
Dual-GFCN-alpha-n,0.808108,0.821569,0.762162,0.597149,0.8071,0.7049
W-GCN,0.762162,0.798039,0.729730,0.473246,0.8061,0.6929


In [21]:
table_acc_val2b

,TexasDataset (0.11),WisconsinDataset (0.20),CornellDataset (0.13),ChameleonDataset (0.24),CoraGraphDataset (0.81),CiteseerGraphDataset (0.74)
Kipf-normA,0.548649,0.468627,0.386486,0.341667,0.8204,0.7084
Kipf-A,0.586486,0.505882,0.483784,0.279825,0.7461,0.6110
MLP,0.759459,0.817647,0.716216,0.461623,0.4948,0.5098
GAT,0.572973,0.507843,0.435135,0.655263,0.7930,0.6960
A-GCN,0.786486,0.825490,0.708108,0.391228,0.8019,0.6935
H-GCN-normH,0.529730,0.462745,0.375676,0.325439,0.8000,0.6855
Dual-GFCN-alpha-n,0.832432,0.825490,0.751351,0.594518,0.8075,0.7018
W-GCN,0.770270,0.776471,0.702703,0.479167,0.8049,0.7151
